In [1]:
!wget https://www.dropbox.com/s/kssgkau7q5ce66z/LC25000.zip?dl=0

--2020-07-10 12:33:50--  https://www.dropbox.com/s/kssgkau7q5ce66z/LC25000.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.1, 2620:100:601d:1::a27d:501
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/kssgkau7q5ce66z/LC25000.zip [following]
--2020-07-10 12:33:50--  https://www.dropbox.com/s/raw/kssgkau7q5ce66z/LC25000.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc131e04132ec0662958d7b94592.dl.dropboxusercontent.com/cd/0/inline/A7TY_gAG_oNJEv2W5l5tT9XCreICFDF21WnBAPI51MYlakjfur28zCsElFhVVsF2AOZOvoewG2RF2ExVP9uSgm8iGqYxOq7CU41eqUZTxZKnofYdF5C36LVCSYMAPZchkKg/file# [following]
--2020-07-10 12:33:51--  https://uc131e04132ec0662958d7b94592.dl.dropboxusercontent.com/cd/0/inline/A7TY_gAG_oNJEv2W5l5tT9XCreICFDF21WnBAPI51MYlakjfur28zCsElFhVVsF2AOZOvoewG2RF2ExVP9uSgm8iGqYxOq7CU41eqUZTxZKnofYd

In [2]:
!pip install spams
!pip install staintools
!pip install plotly
!pip install tqdm
!pip install ipywidgets
!pip install image-quality

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import zipfile
import staintools
import random
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from tqdm import tqdm, tqdm_notebook
#tqdm_notebook().pandas()


#De-noising
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage import img_as_ubyte, img_as_float
from skimage import io

#Image Quality:
import imquality.brisque as brisque


#Threads
import concurrent.futures

#Time
import time

#Warnings
import warnings

%matplotlib inline

In [4]:
BASE_PATH = '/content'

def extract_dataset(path):
  #Extracting the dataset from the zip file

  zip_ref = zipfile.ZipFile(path, 'r')
  zip_ref.extractall(BASE_PATH)
  zip_ref.close()

In [5]:
extract_dataset('LC25000.zip?dl=0')

In [6]:
warnings.filterwarnings("ignore")

In [7]:
path = './lung_colon_image_set/lung_image_sets/'
dir_lst=[]
for i in tqdm(os.listdir(path)):
    if os.path.isdir(path+i):
        dir_lst.append(i)
dir_lst

100%|██████████| 3/3 [00:00<00:00, 8245.68it/s]


['lung_n', 'lung_aca', 'lung_scc']

In [8]:
dir_lst

['lung_n', 'lung_aca', 'lung_scc']

In [9]:
brisque_score_old = {'lung_aca': [],'lung_scc':[],'lung_n':[]}

In [10]:
col_name = ['Image Class','Image File','Brisque Score']
brisque_score_raw = pd.DataFrame(columns=col_name)

In [11]:
brisque_score_raw

,Image Class,Image File,Brisque Score


In [12]:
path='./lung_colon_image_set/lung_image_sets/'#os.listdir(path+dirs)
def calc_brisque_score_n(dirs):
  for i in tqdm(range(len(os.listdir(path+dirs)))):
        imgPath = path+dirs+'/'+os.listdir(path+dirs)[i]
        img_f = img_as_float(io.imread(imgPath))
        #print(img_f)
        row = [{"Image Class":dirs, "Image File	":os.listdir(path+dirs)[i],"Brisque Score":brisque.score(img_f)}]
        #print(row)
        brisque_score_raw.loc[len(brisque_score_raw.index)]=list(row[0].values())
        #brisque.score(img_f)
        

In [13]:
calc_brisque_score_n('lung_n')

100%|██████████| 5000/5000 [6:13:13<00:00,  4.48s/it]


In [14]:
brisque_score_raw

,Image Class,Image File,Brisque Score
0,lung_n,lungn1933.jpeg,50.143688
1,lung_n,lungn2793.jpeg,43.870448
2,lung_n,lungn2701.jpeg,45.603386
3,lung_n,lungn4921.jpeg,46.680690
4,lung_n,lungn4135.jpeg,51.931719
...,...,...,...
4995,lung_n,lungn1190.jpeg,47.827295
4996,lung_n,lungn3876.jpeg,46.463497
4997,lung_n,lungn2910.jpeg,47.993365
4998,lung_n,lungn3645.jpeg,50.178818


In [16]:
brisque_score_raw.to_csv('brisque_score_raw_lung_n.csv', index = False)